In [1]:
# This notebook tests performs basic analytics query to test whether the data was loaded correctly

In [2]:
import configparser
import psycopg2
import boto3
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [3]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

DWH_CLUSTER_TYPE = config.get("DWH", "DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH", "DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH", "DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH", "DWH_DB")
DWH_DB_USER = config.get("DWH", "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH", "DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH", "DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

redshift = boto3.client('redshift',
                        region_name='us-west-2',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                        )

iam = boto3.client('iam',
                   region_name='us-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                   )

roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

In [4]:
# Creating sql engine to query databse by creating connection string 
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
engine = create_engine(conn_string)

In [5]:
# Songplay table query
songplay_df = pd.read_sql_query(text("SELECT songplay_id, user_id, song_id, artist_id, location FROM songplay WHERE session_id = 987"), engine)
songplay_df.head()

,songplay_id,user_id,song_id,artist_id,location
0,5,49,SOCHRXB12A8AE48069,ARTDQRC1187FB4EFD4,"San Francisco-Oakland-Hayward, CA"
1,236,49,SOHGMJH12AB0187130,ARDQ9AP1187B9BA3F6,"San Francisco-Oakland-Hayward, CA"


In [6]:
# User table query
user_df = pd.read_sql_query(text("SELECT user_id, first_name, last_name, gender FROM users WHERE user_id = 53"), engine)
user_df.head()

,user_id,first_name,last_name,gender
0,53,Celeste,Williams,F


In [7]:
# Song table query
song_df = pd.read_sql_query(text("SELECT * FROM songs LIMIT 5"), engine)
song_df.head()

,song_id,title,artist_id,year,duration
0,SONDNMO12A8AE47688,Dance The Night Away,ARSPGHQ1187B9AF447,2003,246.38649
1,SOOHIUW12AB018077F,Learn How To Disappear,ARBR4UG1187B9B5A9D,2006,211.56526
2,SOUYCAY12AB0187B24,Busy Body,AR7I2YB1187B9A88E0,1991,152.34567
3,SOEJCDM12AB017C3F5,I Don't Have To Live This Way,ARF2SVO1187FB53E8F,2004,123.19302
4,SOEXSUW12A8C143796,All Wey A Gwane,ARJQLMQ1187B98DB06,0,235.80689


In [8]:
# Artist table query
artist_df = pd.read_sql_query(text("SELECT * FROM artists LIMIT 5"), engine)
artist_df.head()

,artist_id,name,location,latitude,longitude
0,ARNJLGR1242078249B,Max Avery Lichtenstein,,NaN,NaN
1,ARRYAR91187FB40ABD,Screaming Mechanical Brain,Minnesota,NaN,NaN
2,ARTDQRC1187FB4EFD4,Black Eyed Peas / Les Nubians / Mos Def,"Los Angeles, CA",NaN,NaN
3,ARK0ZK51187FB40BAE,Neil Norman,,NaN,NaN
4,ARASYMJ1187B9ACAF2,Whitesnake,"London, England",51.50632,-0.12714


In [9]:
# Time table query
time_df = pd.read_sql_query(text("SELECT * FROM time LIMIT 5"), engine)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-04 02:54:52.796,2,4,44,11,2018,0
1,2018-11-04 09:31:27.796,9,4,44,11,2018,0
2,2018-11-04 09:51:23.796,9,4,44,11,2018,0
3,2018-11-04 09:59:08.796,9,4,44,11,2018,0
4,2018-11-04 10:05:28.796,10,4,44,11,2018,0
